![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [1]:
import pandas as pd
import numpy as np

# Start coding here...

#read in data file bank_marketing.csv
bank_marketing_df = pd.read_csv('bank_marketing.csv')

#preview data
bank_marketing_df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


In [2]:
#view info about data frame
bank_marketing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [3]:
bank_marketing_clean = bank_marketing_df.copy()

#replace '.' to '_' in job column
bank_marketing_clean['job'] = bank_marketing_clean['job'].replace('.', '_')

bank_marketing_clean['education'] = bank_marketing_clean['education'].replace('.', '_')
bank_marketing_clean['education'] = bank_marketing_clean['education'].replace('unkownn', np.NaN)


In [4]:
bool_cols = ['credit_default', 'mortgage', 'previous_outcome',
             'campaign_outcome', 'campaign_outcome']

#show uniqe values in each bool column
for col in bool_cols:
    print(f"{col}: ", bank_marketing_clean[col].unique())

credit_default:  ['no' 'unknown' 'yes']
mortgage:  ['no' 'yes' 'unknown']
previous_outcome:  ['nonexistent' 'failure' 'success']
campaign_outcome:  ['no' 'yes']
campaign_outcome:  ['no' 'yes']


In [5]:
bool_dict = {'no':False, 'yes':True, 'failure':False, 'success':True, 'nonexistent':False }

for col in bool_cols:
    bank_marketing_clean[col] = bank_marketing_clean[col].map(bool_dict)
    bank_marketing_clean[col] = bank_marketing_clean[col].astype(bool)

In [6]:
bank_marketing_clean['year'] = str(2022)

day = bank_marketing_clean['day'].values.astype(str)
month = bank_marketing_clean['month'].values
year = bank_marketing_clean['year'].values

date = year + "-" + month + "-" + day 
date_format = '%Y-%b-%d'

bank_marketing_clean['last_contact_date'] = pd.to_datetime(date, format=date_format) 

In [7]:
bank_marketing_clean.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome,year,last_contact_date
0,0,56,housemaid,married,basic.4y,False,False,may,13,261,1,0,False,93.994,4.857,True,2022,2022-05-13
1,1,57,services,married,high.school,True,False,may,19,149,1,0,False,93.994,4.857,True,2022,2022-05-19
2,2,37,services,married,high.school,False,True,may,23,226,1,0,False,93.994,4.857,True,2022,2022-05-23
3,3,40,admin.,married,basic.6y,False,False,may,27,151,1,0,False,93.994,4.857,True,2022,2022-05-27
4,4,56,services,married,high.school,False,False,may,3,307,1,0,False,93.994,4.857,True,2022,2022-05-03


In [8]:
bank_marketing_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   age                         41188 non-null  int64         
 2   job                         41188 non-null  object        
 3   marital                     41188 non-null  object        
 4   education                   41188 non-null  object        
 5   credit_default              41188 non-null  bool          
 6   mortgage                    41188 non-null  bool          
 7   month                       41188 non-null  object        
 8   day                         41188 non-null  int64         
 9   contact_duration            41188 non-null  int64         
 10  number_contacts             41188 non-null  int64         
 11  previous_campaign_contacts  41188 non-null  int64     

In [53]:
# split bank_marketing_clean

campaign_df = bank_marketing_clean[['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome', 'last_contact_date']]

client_df = bank_marketing_clean[['client_id', 'age', 'job', 'marital', 'education',
                                 'credit_default', 'mortgage']]

economics_df = bank_marketing_clean[['client_id', 'cons_price_idx', 'euribor_three_months']]

#save dataframes as csv
campaign_df.to_csv('campaign.csv', index=False)
economics_df.to_csv('economics.csv', index=False)
client_df.to_csv('client.csv', index=False)